In [15]:
import numpy as np
import pandas as pd
import os
import datetime
import itertools
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import random
import tensorflow as tf
import math
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import sys
from sklearn import preprocessing
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn import linear_model, tree, ensemble
from xgboost import plot_importance
import optuna
from sklearn.metrics import mean_squared_error as acc
import gc
from lightgbm import LGBMClassifier as lgb
from xgboost import XGBClassifier as clf
from scipy.stats import loguniform
from sklearn import linear_model, tree, ensemble
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier as cat
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import OrdinalEncoder

In [16]:
!pip3 install openpyxl

In [17]:
df=pd.read_excel('../input/dempor/CaseCountData.xlsx')
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [18]:
df.head()

,County,Confirmed Cases,Probable Cases,Fatalities
1,Anderson,4954,2774,171
2,Andrews,2637,0,52
3,Angelina,6546,6656,356
4,Aransas,1920,556,56
5,Archer,937,140,14


In [19]:
df.isna().sum().sort_values(ascending=False).head()

0
Confirmed Cases    2
Probable Cases     2
County             0
Fatalities         0
dtype: int64

In [20]:
df.isna().sum(axis=1).sort_values(ascending=False).head()

256    2
255    2
176    0
163    0
164    0
dtype: int64

**So it has two rows with missing values so we can simply drop them**

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 1 to 256
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   County           256 non-null    object
 1   Confirmed Cases  254 non-null    object
 2   Probable Cases   254 non-null    object
 3   Fatalities       256 non-null    object
dtypes: object(4)
memory usage: 8.1+ KB


In [22]:
df[df.isnull().any(axis=1)]

,County,Confirmed Cases,Probable Cases,Fatalities
255,Unknown,NaN,NaN,23
256,Grand Total,NaN,NaN,63245


In [23]:
df=df.dropna()

In [24]:
print(len(df['County'].unique()),len(df))
#dropping null would be a good option here cause imputing would not add any value to data

254 254


In [25]:
enc=OrdinalEncoder()
df[['County']]=enc.fit_transform(df[['County']])
for i in df.columns:
    df[i]=df[i].astype(float)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 1 to 254
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   County           254 non-null    float64
 1   Confirmed Cases  254 non-null    float64
 2   Probable Cases   254 non-null    float64
 3   Fatalities       254 non-null    float64
dtypes: float64(4)
memory usage: 9.9 KB


In [27]:
df.sort_values('County')

,County,Confirmed Cases,Probable Cases,Fatalities
1,0.0,4954.0,2774.0,171.0
2,1.0,2637.0,0.0,52.0
3,2.0,6546.0,6656.0,356.0
4,3.0,1920.0,556.0,56.0
5,4.0,937.0,140.0,14.0
...,...,...,...,...
250,249.0,3325.0,2764.0,187.0
251,250.0,1131.0,0.0,28.0
252,251.0,2332.0,310.0,58.0
253,252.0,1665.0,502.0,41.0


In [31]:
df.head(256)

,County,Confirmed Cases,Probable Cases,Fatalities
1,0.0,4954.0,2774.0,171.0
2,1.0,2637.0,0.0,52.0
3,2.0,6546.0,6656.0,356.0
4,3.0,1920.0,556.0,56.0
5,4.0,937.0,140.0,14.0
...,...,...,...,...
250,249.0,3325.0,2764.0,187.0
251,250.0,1131.0,0.0,28.0
252,251.0,2332.0,310.0,58.0
253,252.0,1665.0,502.0,41.0


In [29]:
df.to_csv('Covid_Related_CaseCountData.csv', index=False)